<a href="https://colab.research.google.com/github/LiangShuLing/TensorFlowLearning/blob/main/jax/jax_model_practice_AI_learning_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import time
import itertools
import numpy as np
import numpy.random as jnp
from jax import jit, grad, random
from jax.experimental import optimizers, stax
from jax.experimental.stax import Dense, Relu, LogSoftmax


In [ ]:
# define the loss function
def loss(params, batch):
  inputs, targets=batch
  logits=predict(params,inputs)
  return -jnp.mean(jnp.sum(logits*targets,axis=1))

def accuracy(params,batch):
  inputs,targets=batch
  target_class=jnp.argmax(targets,axis=1)
  predicted_class=jnp.argmax(predict(params,inputs),axis=1)
  return jnp.mean(predicted_class==target_class)

# define the network model
init_random_params, preict=stax.serial(
    Dense(25),Relu,
    Dense(12),Relu,
    Dense(6),LogSoftmax)



In [ ]:
def main_func():
  rng=random.PRNGKey(0)

  step_size=0.001
  num_epochs=10
  batch_size=128
  momentum_mass=0.9

#load data
  train_images, train_labels, test_images, test_labels = datasets
  num_train=train_images.shape[0]
  input_shape=(train_images[0].shape, batch_size)
  
  num_complete_batches, leftover=divmod(num_train,batch_size)  # python divmod() 函数把除数和余数运算结果结合起来，返回一个包含商和余数的元组(a // b, a % b)。
  #返回完整的batch个数，以及剩下的train data
  num_batches=num_complete_batches+bool(leftover)   #如果还有剩下data, batch数加一，否则加0

  def data_stream():
    rng=npr.RandomState(0)

    while True:
      perm=rng.permutation(num_train)  # 随机排列一个序列，或者数组。如果x是多维数组，则沿其第一个坐标轴的索引随机排列数组。
      for i in range(num_batches):
        batch_idx=perm[i*batch_size:(i+1)*batch_size]   #按batch_size来取数据的索引
        yield train_images[batch_idx],train_labels[batch_idx]  #按照索引来获取训练数据与标签

  batches=data_stream()

  opt_init,opt_update, get_params=optimizers.adam(step_size,eps=0.0001)  #使用Adam优化器

  @jit
  def update(i,opt_state,batch):
    params=get_params(opt_state)
    return opt_update(i,grad(loss)(params,batch),opt_state)
  
  _, init_params=init_random_params(rng,input_shape)
  opt_state=opt_init(init_params)
  itercount=itertools.count()

  print('\nStarting training...')
  for epoch in range(num_epoches):
    start_time=time.time()

    for _ in range(num_batches):
      opt_state=update(next(itercount),opt_state,next(batches))
    epoch_time=time.time()-start_time

    params=get_params(opt_state)
    train_acc=accuracy(params,(train_images,train_labels))
    test_acc=accuracy(params,(test_images,test_labels))
    print("Epoch {} in {:0.2f} sec".format(epoch,epoch_time))
    print("Training set accuracy {}".format(train_acc))
    print("Test set accuracy {}".format(test_acc))
    